<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [30]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [31]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [32]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [33]:
# use requests.get() method with the provided static_url
# assign the response to a object
#me
url='https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'
#url = url + '/?page={0}'.format(i)
print(url)
r=requests.get(url)
soup=BeautifulSoup(r.text,'html.parser')

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


Create a `BeautifulSoup` object from the HTML `response`


In [ ]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [34]:
# Use soup.title attribute
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [35]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

In [36]:
len(html_tables)

19

Starting from the third table is our target table contains the actual launch records.


In [37]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table.prettify())

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     booster
    </a>
    <sup class="reference" id="cite_ref-booster_16-0">
     <a href="#cite_note-booster-16">
      [a]
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch
    <br/>
    site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_17-0">
     <a href="#cite_note-Dragon-17">
      [b]
     </a>
    </sup>
   </th>
   <th scope="col">
    Payload mass
   </th>
   <th scope="col">
    Orbit
   </th>
   <th scope="col">
    Customer
   </th>
   <th scope="col">
    Launch
    <br/>


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [38]:
print(url)
r=requests.get(url)
soup=BeautifulSoup(r.text,'html.parser') 
names=soup.find_all('div',attrs={'data-test':"cardContent"})

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


In [39]:
column_names = []
population_data = pd.DataFrame(columns=["Date", "version", "Launch", "Payload", "Payload_mass"])

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for row in first_launch_table.tbody.find_all('tr'):
    #extract_column_from_header(row)
    #print(row.text)
    col = row.find_all('td',colspan=False)
    #col1=row.find_all('td',attrs={'colspan':'9'})
    if (col != []):
       # if col1 is not None:
            Date = col[0].text
            version = col[1].text
            Launch = col[2].text.strip()
            Payload = col[3].text.strip()
            Payload_mass = col[4].text.strip()
            Orbit = col[5].text.strip()
            Customer = col[6].text.strip()
            Launchoutcome = col[7].text.strip()
            Boosterlanding = col[8].text.strip()
            population_data = population_data.append({"Date":Date, "version":version, "Launch":Launch, "Payload":Payload, "Payload_mass":Payload_mass,'Orbit':Orbit,'Customer':Customer,'Launchoutcome':Launchoutcome,'Boosterlanding':Boosterlanding}, ignore_index=True)

population_data

C:\Users\arash\AppData\Local\Temp\ipykernel_12316\3349744458.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  population_data = population_data.append({"Date":Date, "version":version, "Launch":Launch, "Payload":Payload, "Payload_mass":Payload_mass,'Orbit':Orbit,'Customer':Customer,'Launchoutcome':Launchoutcome,'Boosterlanding':Boosterlanding}, ignore_index=True)
C:\Users\arash\AppData\Local\Temp\ipykernel_12316\3349744458.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  population_data = population_data.append({"Date":Date, "version":version, "Launch":Launch, "Payload":Payload, "Payload_mass":Payload_mass,'Orbit':Orbit,'Customer':Customer,'Launchoutcome':Launchoutcome,'Boosterlanding':Boosterlanding}, ignore_index=True)
C:\Users\arash\AppData\Local\Temp\ipykernel_12316\3349744458.py:23: FutureWarning: T

,Date,version,Launch,Payload,Payload_mass,Orbit,Customer,Launchoutcome,Boosterlanding
0,7 January 202002:19:21[13]\n,F9 B5 ♺ B1049.4\n,"CCSFS,SLC-40",Starlink 2 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Success(drone ship)
1,19 January 202015:30[16]\n,F9 B5 ♺ B1046.4\n,"KSC,LC-39A",Crew Dragon in-flight abort test[17](Dragon C2...,"12,050 kg (26,570 lb)",Sub-orbital[18],NASA (CTS)[19],Success,No attempt
2,29 January 202014:07[24]\n,F9 B5 ♺ B1051.3\n,"CCSFS,SLC-40",Starlink 3 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Success(drone ship)
3,17 February 202015:05[26]\n,F9 B5 ♺ B1056.4\n,"CCSFS,SLC-40",Starlink 4 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Failure(drone ship)
4,7 March 202004:50[29]\n,F9 B5 ♺ B1059.2\n,"CCSFS,SLC-40",SpaceX CRS-20(Dragon C112.3 ♺ ),"1,977 kg (4,359 lb)[30] (excl. Dragon mass)",LEO (ISS),NASA (CRS),Success,Success(ground pad)
5,18 March 202012:16[33]\n,F9 B5 ♺ B1048.5\n,"KSC,LC-39A",Starlink 5 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Failure(drone ship)
6,22 April 202019:30[37]\n,F9 B5 ♺ B1051.4\n,"KSC,LC-39A",Starlink 6 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Success(drone ship)
7,30 May 202019:22[40]\n,F9 B5 B1058.1[41]\n,"KSC,LC-39A",Crew Dragon Demo-2[42](Crew Dragon C206.1 Ende...,"12,530 kg (27,620 lb)[43]",LEO (ISS),NASA (CCDev),Success,Success(drone ship)
8,4 June 202001:25[45]\n,F9 B5 ♺ B1049.5\n,"CCSFS,SLC-40",Starlink 7 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Success(drone ship)
9,13 June 202009:21[47]\n,F9 B5 ♺ B1059.3\n,"CCSFS,SLC-40","Starlink 8 v1.0 (58 satellites),[48][49]SkySat...","15,410 kg (33,970 lb)[47]",LEO,SpaceXPlanet Labs,Success,Success(drone ship)


In [87]:
type(population_data)

pandas.core.frame.DataFrame

In [44]:
col = row.find_all('td',colspan=False)
col

[]

In [21]:
population_data.columns

Index(['Date', 'version', 'Launch', 'Payload', 'Payload_mass', 'Orbit',
       'Customer', 'Launchoutcome', 'Boosterlanding'],
      dtype='object')

In [45]:
#me pandas html 
url='https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'

read_html_pandas_data = pd.read_html(url)

In [49]:
#me
read_html_pandas_data[2]

,Flight No.,Date and time (UTC),"Version, booster[a]",Launch site,Payload[b],Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,78,7 January 2020 02:19:21[13],F9 B5 ♺ B1049.4,"CCSFS, SLC-40",Starlink 2 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Success (drone ship)
1,78,Third large batch and second operational fligh...,Third large batch and second operational fligh...,Third large batch and second operational fligh...,Third large batch and second operational fligh...,Third large batch and second operational fligh...,Third large batch and second operational fligh...,Third large batch and second operational fligh...,Third large batch and second operational fligh...,Third large batch and second operational fligh...
2,79,19 January 2020 15:30[16],F9 B5 ♺ B1046.4,"KSC, LC-39A",Crew Dragon in-flight abort test[17] (Dragon C...,"12,050 kg (26,570 lb)",Sub-orbital[18],NASA (CTS)[19],Success,No attempt
3,79,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...,An atmospheric test of the Dragon 2 abort syst...
4,80,29 January 2020 14:07[24],F9 B5 ♺ B1051.3,"CCSFS, SLC-40",Starlink 3 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Success (drone ship)
5,80,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...,Third operational and fourth large batch of St...
6,81,17 February 2020 15:05[26],F9 B5 ♺ B1056.4,"CCSFS, SLC-40",Starlink 4 v1.0 (60 satellites),"15,600 kg (34,400 lb)[14]",LEO,SpaceX,Success,Failure (drone ship)
7,81,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...,Fourth operational and fifth large batch of St...
8,82,7 March 2020 04:50[29],F9 B5 ♺ B1059.2,"CCSFS, SLC-40",SpaceX CRS-20 (Dragon C112.3 ♺ ),"1,977 kg (4,359 lb)[30] (excl. Dragon mass)",LEO (ISS),NASA (CRS),Success,Success (ground pad)
9,82,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...,Last launch of phase 1 of the CRS contract. Ca...


Check the extracted column names


In [24]:
column_names=['Date', 'version', 'Launch', 'Payload', 'Payload_mass', 'Orbit',
       'Customer', 'Launchoutcome', 'Boosterlanding']

In [25]:
print(column_names)

['Date', 'version', 'Launch', 'Payload', 'Payload_mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [26]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

KeyError: 'Date and time ( )'

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [27]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5B1058.1
F9 B5
F9 B5
F9 B5
F9 B5B1058.2
F9 B5
F9 B5B1049.6
F9 B5
F9 B5B1060.2
F9 B5B1058.3
F9 B5B1051.6
F9 B5
F9 B5
F9 B5B1061.1
F9 B5
F9 B5B1049.7
F9 B5B1058.4
F9 B5
F9 B5
F9 B5
F9 B5B1051.8
F9 B5B1058.5


AttributeError: 'NoneType' object has no attribute 'string'

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [28]:
df=pd.DataFrame(launch_dict)

ValueError: If using all scalar values, you must pass an index

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
